In [2]:
import numpy as np                                          
from mindquantum.core.circuit import Circuit                
from mindquantum.core.gates import H, RX, RY, RZ,X    
from mindquantum.core.parameterresolver import PRGenerator  
from mindquantum.simulator import Simulator
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split   
from mindquantum.algorithm.library import amplitude_encoder
from mindquantum.algorithm.nisq import HardwareEfficientAnsatz     
from mindquantum.core.operators import QubitOperator           # 导入QubitOperator模块，用于构造泡利算符
from mindquantum.core.operators import Hamiltonian             # 导入Hamiltonian模块，用于构建哈密顿量
import mindspore as ms                                                                         # 导入mindspore库并简写为ms
from mindquantum.framework import MQLayer                                                      # 导入MQLayer
# 导入HardwareEfficientAnsatz
from mindquantum.core.gates import RY           
import torch
from torchvision import datasets, transforms# 导入量子门RY
from scipy.ndimage import zoom
import random

In [3]:
# 定义数据预处理
transform = transforms.Compose([
    transforms.ToTensor()
])
np.random.seed(10)
def filter_3_and_6(data):
    images, labels = data
    mask = (labels == 3) | (labels == 6)
    return images[mask], labels[mask]

# 下载和加载 MNIST 数据集
mnist_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=None)
filtered_data = filter_3_and_6((mnist_dataset.data, mnist_dataset.targets))
X_data, y = filtered_data  # X 图像数据 y 标签

# 分别对标签为 3 和 6 的数据进行随机抽样
def sample_data(X, y, label, sample_ratio=0.2):
    label_mask = (y == label)
    X_label = X[label_mask]
    y_label = y[label_mask]
    
    sample_size = int(len(y_label) * sample_ratio)
    sample_indices = np.random.choice(len(y_label), sample_size, replace=False)
    
    return X_label[sample_indices], y_label[sample_indices]

X_data_3, y_data_3 = sample_data(X_data, y, label=3, sample_ratio=0.2)
X_data_6, y_data_6 = sample_data(X_data, y, label=6, sample_ratio=0.2)

# 合并抽样后的数据
X_sampled = torch.cat((X_data_3, X_data_6), dim=0)
y_sampled = torch.cat((y_data_3, y_data_6), dim=0)

# 打印抽样后的数据形状以验证结果
print(X_sampled.shape, y_sampled.shape)

torch.Size([2409, 28, 28]) torch.Size([2409])


In [ ]:
def amplitude_param(pixels):
    param_rd = []
    _, parameterResolver = amplitude_encoder(pixels, 8)   
    for _, param in parameterResolver.items():
        param_rd.append(param)
    param_rd = np.array(param_rd)
    return param_rd

# 将幅度转为编码线路参数，幅度shape(256,)，参数shape(255,)
train_param = np.array([amplitude_param(i) for i in train_pixels ])
test_param = np.array([amplitude_param(i) for i in test_pixels ])